Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [19]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [20]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48817 entries, 0 to 49351
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bathrooms             48817 non-null  float64
 1   bedrooms              48817 non-null  int64  
 2   created               48817 non-null  object 
 3   description           47392 non-null  object 
 4   display_address       48684 non-null  object 
 5   latitude              48817 non-null  float64
 6   longitude             48817 non-null  float64
 7   price                 48817 non-null  int64  
 8   street_address        48807 non-null  object 
 9   interest_level        48817 non-null  object 
 10  elevator              48817 non-null  int64  
 11  cats_allowed          48817 non-null  int64  
 12  hardwood_floors       48817 non-null  int64  
 13  dogs_allowed          48817 non-null  int64  
 14  doorman               48817 non-null  int64  
 15  dishwasher         

In [22]:
df.shape

(48817, 34)

In [23]:
df['display_address'].value_counts()[df['display_address'].value_counts() <= 5].index

Index(['Monitor Street', 'W 55 St.', '1403 New York Avenue', 'W 89th St.',
       'E 88th', 'Bay Pkwy', '154A Washington Ave', 'Berry St.',
       '50 West 34th Street', '86-02 Forest Pkwy',
       ...
       '166 Orchard st', '14010 Franklin Avenue', 'wall st.', 'W. 31st Street',
       ' Park Ave', '332 west 37th Street', 'williamsburg  NO FEE ', 'W 191st',
       'LUXURY TWO BED HOME.......NO FEE!!!  AMAZING DEAL!!!!',
       '56th & Lexington Ave'],
      dtype='object', length=7356)

In [24]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# cleaning and reshaping dataset
clean = df.copy()
clean['has_description'] = ((clean['description'].isna()==False) & (clean['description'].str.strip().str.len() > 0)).replace({False: 0, True: 1})
clean['created_dt'] = pd.to_datetime(clean['created'])

clean['created_week'] = clean['created_dt'].dt.weekofyear

# Replacing the column 'interest_lvl' from obj to int.
clean['interest_numeric'] = clean['interest_level'].replace({'low':1,'medium':2,'high':3})

# clean['is_broadway'] = clean['display_address']=='Broadway'
clean['display_address'] = clean['display_address'].str.strip().str.lower()
top_addresses = list(clean['display_address'].value_counts()[clean['display_address'].value_counts() >= 15].index)
clean['top_addresses'] = clean['display_address'].where(clean['display_address'].isin(top_addresses), other='other')
clean = clean.join(pd.get_dummies(clean['top_addresses'], prefix='address_'))


In [26]:
# Dsiplaying the top adrresses
clean['top_addresses'].value_counts()

other               16030
broadway              439
east 34th street      355
second avenue         351
wall street           336
                    ...  
e 16th st.             15
w 77 street            15
e 33 st.               15
east 48th street       15
dekalb ave.            15
Name: top_addresses, Length: 676, dtype: int64

In [27]:
# Date and time
clean['created_dt'].dt.month

0        6
1        6
2        4
3        4
4        4
        ..
49347    6
49348    4
49349    4
49350    4
49351    4
Name: created_dt, Length: 48817, dtype: int64

In [35]:
import sklearn.model_selection as model_selection

# Using data from april and may for train n june for test
train = clean[(clean['created_dt'].dt.month==4) | (clean['created_dt'].dt.month==5)]
test = clean[clean['created_dt'].dt.month==6]

In [30]:
## Arranging the target and feature matrix n vector
target = 'price'
features = clean.columns[clean.dtypes!='object']
features = features.drop(target)
features = features.drop('created_dt')

In [32]:
from sklearn.linear_model import LinearRegression
import numpy

# Instantiating the class
lr_model = LinearRegression()
# Fitting the model to the training data
lr_model.fit(train[features],train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [37]:
# model coef
lr_model.coef_[0]

1554.2938564162596

In [38]:
# model intercept
lr_model.intercept_

-1054437.3808086866

In [42]:
import sklearn.metrics as metrics

y_train = lr_model.predict(train[features])
y_test = lr_model.predict(test[features])

# Getting regression metric of rmse, mae, n r**2 for train
train_rmse = numpy.sqrt(metrics.mean_squared_error(train[target], y_train))
train_mae = metrics.mean_absolute_error(train[target], y_train)
train_r2 = metrics.r2_score(train[target], y_train)

# Getting regression metric of rmse, mae, n r**2 for test
test_rmse = numpy.sqrt(metrics.mean_squared_error(test[target], y_test))
test_mae = metrics.mean_absolute_error(test[target], y_test)
test_r2 = metrics.r2_score(test[target], y_test)

# Baseline MAE test
mean = numpy.mean(clean[target])
baseline_rmse = numpy.sqrt(metrics.mean_squared_error(clean[target],numpy.linspace(mean, mean, len(clean[target]))))
baseline_mae = metrics.mean_absolute_error(clean[target], numpy.linspace(mean, mean, len(clean[target])))
baseline_r2 = metrics.r2_score(clean[target],numpy.linspace(mean, mean, len(clean[target])))

print(f'Features: \n{features}\n')
print(f'Baseline Root Mean Squared Error: {baseline_rmse}')
print(f'Baseline Mean Absolute Error: {baseline_mae}')
print(f'Baseline R^2 score: {baseline_r2}')
print(f'Train Root Mean Squared Error: {train_rmse}')
print(f'Train Mean Absolute Error: {train_mae}')
print(f'Train R^2 score: {train_r2}')
print(f'Test Root Mean Squared Error: {test_rmse}')
print(f'Test Mean Absolute Error: {test_mae}')
print(f'Test R^2 score: {test_r2}')

Features: 
Index(['bathrooms', 'bedrooms', 'latitude', 'longitude', 'elevator',
       'cats_allowed', 'hardwood_floors', 'dogs_allowed', 'doorman',
       'dishwasher',
       ...
       'address__west houston street', 'address__west st', 'address__west st.',
       'address__west street', 'address__williamsburg', 'address__worth',
       'address__worth street', 'address__york ave', 'address__york ave.',
       'address__york avenue'],
      dtype='object', length=707)

Baseline Root Mean Squared Error: 1762.4127206231178
Baseline Mean Absolute Error: 1201.532252154329
Baseline R^2 score: 0.0
Train Root Mean Squared Error: 960.0093212890665
Train Mean Absolute Error: 603.0311325133185
Train R^2 score: 0.7031853392985251
Test Root Mean Squared Error: 963.7690571199786
Test Mean Absolute Error: 626.3262220640404
Test R^2 score: 0.701144002755262
